### Задание 1
взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)

https://archive.ics.uci.edu/ml/datasets/Early+stage+diabetes+risk+prediction+dataset.

Early stage diabetes risk prediction dataset. Data Set

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('diabetes_data_upload.csv')
df.head(3)

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive


### Задание 2
сделать feature engineering

In [10]:
df.replace('Yes', 1, inplace=True)
df.replace('Positive', 1, inplace=True)
df.replace('No', 0, inplace=True)
df.replace('Negative', 0, inplace=True)

df.replace('Male', 0, inplace=True)
df.replace('Female', 1, inplace=True)

df.head(3)

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,0,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1
1,58,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1
2,41,0,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1


In [11]:
df.isna().sum()

Age                   0
Gender                0
Polyuria              0
Polydipsia            0
sudden weight loss    0
weakness              0
Polyphagia            0
Genital thrush        0
visual blurring       0
Itching               0
Irritability          0
delayed healing       0
partial paresis       0
muscle stiffness      0
Alopecia              0
Obesity               0
class                 0
dtype: int64

In [12]:
df.nunique()

Age                   51
Gender                 2
Polyuria               2
Polydipsia             2
sudden weight loss     2
weakness               2
Polyphagia             2
Genital thrush         2
visual blurring        2
Itching                2
Irritability           2
delayed healing        2
partial paresis        2
muscle stiffness       2
Alopecia               2
Obesity                2
class                  2
dtype: int64

In [17]:
target = 'class'
features = df.drop(columns=[target]).columns.tolist()

target, features

('class',
 ['Age',
  'Gender',
  'Polyuria',
  'Polydipsia',
  'sudden weight loss',
  'weakness',
  'Polyphagia',
  'Genital thrush',
  'visual blurring',
  'Itching',
  'Irritability',
  'delayed healing',
  'partial paresis',
  'muscle stiffness',
  'Alopecia',
  'Obesity'])

Соотношение классов

In [19]:
df[target].value_counts()

1    320
0    200
Name: class, dtype: int64

### Задание 3
обучить любой классификатор (какой вам нравится)

In [37]:
random_state = 20

from sklearn.model_selection import train_test_split

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=random_state)

In [38]:
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=random_state)

model.fit(X_train, y_train)
y_predict = model.predict(X_test)

Проверяем качество

In [39]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 98.26%
roc: 97.36%
recall: 99.12%
precision: 97.41%


### Задание 4
далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть

In [40]:
mod_df = df.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_df[target].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 80/320 as positives and unlabeling the rest


In [42]:
mod_df['class_test'] = -1
mod_df.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_df.iloc[:,-1].value_counts())

target variable:
 -1    440
 1     80
Name: class_test, dtype: int64


In [44]:
x_data = mod_df.iloc[:,:-2].values # just the X 
y_labeled = mod_df.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_df.iloc[:,-2].values # original class

### Задание 5
применить random negative sampling для построения классификатора в новых условиях

In [45]:
mod_df = mod_df.sample(frac=1)
neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])]
sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):]
pos_sample = mod_df[mod_df['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(80, 18) (80, 18)


In [47]:
model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

### Задание 6
сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)

In [49]:
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 90.64%
roc: 88.99%
recall: 95.83%
precision: 85.98%


### Задание 7
поэкспериментировать с долей P на шаге 5(4) (как будет меняться качество модели при уменьшении/увеличении размера P)

In [57]:
for s in np.arange(0.15, 0.40, 0.05):
    print(f'\npositives marked size: {s}')
    mod_df = df.copy()
    #get the indices of the positives samples
    pos_ind = np.where(mod_df[target].values == 1)[0]
    #shuffle them
    np.random.shuffle(pos_ind)
    # leave just 25% of the positives marked
    pos_sample_len = int(np.ceil(s * len(pos_ind)))
    print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
    pos_sample = pos_ind[:pos_sample_len]
    
    mod_df['class_test'] = -1
    mod_df.loc[pos_sample,'class_test'] = 1
    print('target variable:\n', mod_df.iloc[:,-1].value_counts())
    
    x_data = mod_df.iloc[:,:-2].values # just the X 
    y_labeled = mod_df.iloc[:,-1].values # new class (just the P & U)
    y_positive = mod_df.iloc[:,-2].values # original class
    
    mod_df = mod_df.sample(frac=1)
    neg_sample = mod_df[mod_df['class_test']==-1][:len(mod_df[mod_df['class_test']==1])]
    sample_test = mod_df[mod_df['class_test']==-1][len(mod_df[mod_df['class_test']==1]):]
    pos_sample = mod_df[mod_df['class_test']==1]
    print(neg_sample.shape, pos_sample.shape)
    sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)
    
    model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
    y_predict = model.predict(sample_test.iloc[:,:-2].values)
    
    evaluate_results(sample_test.iloc[:,-2].values, y_predict)


positives marked size: 0.15
Using 48/320 as positives and unlabeling the rest
target variable:
 -1    472
 1     48
Name: class_test, dtype: int64
(48, 18) (48, 18)
Classification results:
f1: 92.40%
roc: 89.93%
recall: 94.80%
precision: 90.11%

positives marked size: 0.2
Using 64/320 as positives and unlabeling the rest
target variable:
 -1    456
 1     64
Name: class_test, dtype: int64
(64, 18) (64, 18)
Classification results:
f1: 94.28%
roc: 93.82%
recall: 92.38%
precision: 96.26%

positives marked size: 0.25
Using 80/320 as positives and unlabeling the rest
target variable:
 -1    440
 1     80
Name: class_test, dtype: int64
(80, 18) (80, 18)
Classification results:
f1: 91.18%
roc: 89.52%
recall: 97.38%
precision: 85.71%

positives marked size: 0.30000000000000004
Using 97/320 as positives and unlabeling the rest
target variable:
 -1    423
 1     97
Name: class_test, dtype: int64
(97, 18) (97, 18)
Classification results:
f1: 95.45%
roc: 94.96%
recall: 97.11%
precision: 93.85%

p